In [ ]:

import torch
from torchvision import models

vgg16 = models.vgg16(pretrained=True)


vgg19 = models.vgg19(pretrained=True)




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 75.5MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future.

In [ ]:
vgg19

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# 데이터셋을 불러옵니다.
train_dataset = MNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = MNIST(root='data', train=False, download=True, transform=transforms.ToTensor())


In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),  # 텐서를 PIL 이미지로 변환
    transforms.Resize((224, 224)),  # VGG19에 맞는 크기로 변경
    transforms.Grayscale(num_output_channels=3),  # 1채널 이미지를 3채널로 변환
    transforms.ToTensor(),  # PIL 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
])

In [ ]:
# DataLoader를 정의합니다.
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
# 모델을 정의하고 옵티마이저 및 손실 함수를 설정합니다.
model = VGG19(num_classes=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
from tqdm import tqdm

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    # tqdm을 사용하여 학습 진행 상태를 표시합니다.
    with tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit='batch') as tepoch:
        for i, (images, labels) in tepoch:
            # 이미지를 3채널로 변환 및 전처리
            images = torch.stack([transform(image.squeeze(0)) for image in images])
            images, labels = images.to(device), labels.to(device)

            # 순전파 및 손실 계산
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 진행 바 업데이트
            tepoch.set_postfix(loss=loss.item())

# 평가 루프
model.eval()
correct = 0
total = 0
# 평가 진행 상태를 표시합니다.
with torch.no_grad(), tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating", unit='batch') as ttest:
    for i, (images, labels) in ttest:
        images = torch.stack([transform(image.squeeze(0)) for image in images])
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


In [ ]:
import torch
import torch.nn as nn

# inplace=True 수행할 때, 추가적인 메모리 할당 없이 입력 데이터를 바로 수정하기 위해 사용
class VGG19(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG19, self).__init__()
        # Convolutional layers
        self.features = nn.Sequential(
            # 1 block
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #128 *2

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #256*4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #512*4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            #512*4
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier=nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )





    def forward(self, x):
        # Apply the feature extractor in the input
        x = self.features(x)
        # Flatten the output of the conv layers
        x = x.view(x.size(0), -1)
        # Apply the classifier on the flattened features
        x = self.classifier(x)
        return x

# Instantiate the model
model = VGG19(num_classes=10)
print(model)
